# Testing

In this link, https://fastapi.tiangolo.com/tutorial/testing, you may have noticed "X-token" in the following paragraph:

```
Let's say that now the file main.py with your FastAPI app has some other path operations.

It has a GET operation that could return an error.

It has a POST operation that could return several errors.

Both path operations require an X-Token header.
```

My understanding of this is *assume* you have a FastAPI app that has e.g. a GET operator or a POST operation that requires *some* token in the header - "X-token" is just a way of saying some token that you may require in every HTTP request. See the subsection "PasswordBearer" in the section "Security" for an example of one of these tokens.

# Security

## PasswordBearer

(Understanding the following link: https://fastapi.tiangolo.com/tutorial/security/first-steps/#__tabbed_1_2)

As a user on the front end (web browser) visiting an endpoint which contains authorisation, you will:
- Enter your username and password and press enter.
- This will send your username and password to a particular endpoint. That endpoint is specified in the code:
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")  
The endpoint is “token” == “./token”, a relative URL. So, if your API is at https://example.com/api/v1/, your username and password will be sent to https://example.com/api/v1/token.  
That endpoint will respond with a token which is just a string e.g. “mysecrettoken”. This is stored in the browser temporarily.  
Note that this `tokenUrl=”token”` doesn’t create that endpoint, it just declares where the client should go to get the token.
-	Now, let’s say the user visits another part of the frontend which requires more data from the API and authentication. 
To authenticate with our API, the request made will contain a header called “Authorisation” with a value of “Bearer” + the token -> “Bearer mysecrettoken”


## Dependencies and Dependency Injection

I am following the tutorials in this playlist: https://www.youtube.com/watch?v=Kq7ezzVInCA

Take a look at the following code that implements pagination to two endpoints:

In [7]:
from fastapi import Depends, FastAPI
from typing import Union

app = FastAPI()

@app.get("/items/")
def read_items(q: Union[str, None] = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}

@app.get("/users/")
def read_items(q: Union[str, None] = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}

As you can see, we are repeating ourselves quite a bit here with those arguments. Wouldn't it be better if we could factor it out? Here's how:

In [9]:
from fastapi import Depends, FastAPI
from typing import Union

app = FastAPI()

def common_parameters(q: Union[str, None] = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}

@app.get("/items/")
async def read_items(commons: dict = Depends(common_parameters)):
    return commons

@app.get("/users/")
def read_items(commons: dict = Depends(common_parameters)):
    return commons

The two above are identical, but the approach using `Depends` is much cleaner. `Depends` takes a callable and calls it. Then, it returns whatever the callable returns to the parameter that it's pointing to.

Run main.py in the fastapi_bench folder to confirm it works.

### Nesting Dependencies

As mentioned right before, `Depends` takes a callable and calls it. The callable can contain `Depends` too, which will be called as well.

In [7]:
def hello():
    return "world"

def common_parameters_3(
    q: Union[str, None] = None,
    skip: int = 0,
    limit: int = 100,
    blah: str = Depends(hello),
):
    return {"q": q, "skip": skip, "limit": limit, "hello": blah}

@app.get("/users_3/")
def read_users_3(commons: dict = Depends(common_parameters_3)):
    return commons

As you can see, we will expect `common_parameters_3` to be called, which contains a `Depends(hello)` so `hello()` is called and returns `"world"`. This string is stored in `blah` which is then returned as part of `common_3_parameters`. See the output of the request below:

```python
{
  "q": null,
  "skip": 0,
  "limit": 100,
  "hello": "world"
}
```

### Classes as Dependencies

Take a look at this example (https://www.youtube.com/watch?v=jJDzJg3O9ZQ):

In [8]:
fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]


class CommonQueryParams:
    def __init__(
        self,
        item_id: int,
        q: Union[str, None] = None,
        skip: int = 0,
        limit: int = 100
    ):
        self.q = q
        self.skip = skip
        self.limit = limit
        self.item_id = item_id


@app.get("/items_4/{item_id}")
def read_items(commons: CommonQueryParams = Depends(CommonQueryParams)):
    response = {}
    print(commons.item_id)
    if commons.q:
        response.update({"q": commons.q})
    items = fake_items_db[commons.skip:commons.skip + commons.limit]
    response.update({"items": items})
    return response


Here's what happens step-by-step when we make the following GET request: http://localhost:8000/items_4/123?skip=1&limit=3

1. FastAPI identifies that a request to `/users_4/` was made with a number of query parameters.
2. It is about to execute the associated function `read_items` before it realises that `commons`'s default value is a dependency.
3. FastAPI looks at the dependency (`CommonQueryParams`) to see what function parameters it expects.
4. In our example, it expects `item_id`, `q`, `skip` and `limit`, though most of these have default values.
5. When FastAPI executes `CommonQueryParams`, it knows which query parameters it needs, so it extracts these from the query parameters in the request and passes them to `CommonQueryParams`
6. It also detects that we need `item_id` which is not a query parameter but a *path* parameter and passes that to `CommonQueryParams` too.
7. `commons` is now equivalent to that class, which is why we have type-hinted it with `CommonQueryParams`.
8. We can access any of this class's attributes like normal.

As a note, FastAPI has numerous shorthands for the following argument of `read_items(commons: CommonQueryParams = Depends(CommonQueryParams))`.

Note that all the below are equivalent:
- `commons: CommonQueryParams = Depends(CommonQueryParams)`
- `commons: CommonQueryParams = Depends()`
- `commons=Depends(CommonQueryParams)`


### `Annotated`

https://fastapi.tiangolo.com/tutorial/query-params-str-validations/

Take a look at this:

In [3]:
from fastapi import FastAPI, Depends, Query
from typing_extensions import Annotated
from typing import Union

app = FastAPI()

@app.get("/others/")
def read_others(q: Annotated[str, Query(min_length=3)]):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

Looking at `q: Annotated[str, Query(min_length=3)]`, since no default has been specified for `q`, `Query(..)`'s return value will become the default.

In [2]:
from fastapi import FastAPI, Depends, Query
from typing_extensions import Annotated
from typing import Union

app = FastAPI()

@app.get("/items_2/")
def read_items_2(q: Annotated[Union[str, None], Query(max_length=50)] = None):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

If you try `GET`ting a string that's over 50 characters long, you will get a HTTP 422 error. 

You might be wondering how `Query` actually validates the input. I am not sure, but this is perhaps a simplified way that it does it:

In [ ]:
from typing import Optional
from fastapi import Request

def Query(param_name: str, default: Optional[str] = None, max_length: Optional[int] = None):
    def query_parameter(request: Request):
        value = request.query_params.get(param_name, default)

        if max_length is not None and value is not None and len(value) > max_length:
            raise ValueError(f"Value of '{param_name}' parameter is too long")
        
        return value

    return query_parameter

To summarise, it seems that it takes the entire request and gets the value of the parameter associated with `q`.

It's **very important** to note that the calling and subsequent validating of `q` via `Query` is implemented in FastAPI not `Annotated` or Python itself. To elaborate, the `Annotated` library simply provides metadata to anyone that wants it. It is up to the user (or FastAPI in this case) to do something with that metadata. 

In our case, FastAPI looks at the HTTP request, e.g. `http://localhost:8000/items_2/?q=somestring`, and finds a query parameter associated with `q`, i.e. `somestring`. 

Then, it stores this into the `q` parameter of the `read_items_2` function.

Then, it checks to see if it has metadata attached to it and recognises immediately that it does.

Then, in FastAPI's internal implementation, it will pass `q` to `Query` for validation, but this is all hidden from the developer. All they need to do, is provide the validation function in `Annotated` of `q` and FastAPI will handle the rest.